In [1]:
import json
import os
import sys

In [2]:
from icecream import ic
import numpy as np
import pandas as pd
from pprint import pprint
import torch
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import densenet
from tqdm import tqdm

In [3]:
os.chdir('../')
sys.path.append('../dataset/')
import coco_data_prep
import coco_api_helper
import config_dataset

loading annotations into memory...
Done (t=12.91s)
creating index...
index created!
loading annotations into memory...
Done (t=5.54s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [4]:
annot_filepaths = {'train': config_dataset.train_annotation_filepath,
                 'validation' : config_dataset.valid_annotation_filepath,
                 'test': config_dataset.test_annotation_filepath}

with open(annot_filepaths['train'], 'r') as f:
    train_annot = json.load(f)

### Create the Cateogries Label file

In [5]:
# cat_ids = [annot['id'] for annot in train_annot['categories']]
# cat_names = [annot['name'] for annot in train_annot['categories']]

# cat_table = pd.DataFrame(pd.Series(cat_ids, cat_names))
# cat_table.reset_index(drop=False, inplace=True)
# cat_table.rename(columns={
#     'index': 0,
#     0: 1
# }, inplace=True)
# cat_table = cat_table[[1, 0]]
# cat_table.to_csv('../dataset/coco_labels.txt', header=None)

### Helper Functions

In [6]:
def calc_new_dim(orig_h_or_w: int, padding, kernel, stride) -> int:
    """
    :orig_h_or_w: original width or original height of the input image
    given the original width or height, kernel size, pad width, stride size
    calculate the new width or height
    """
    new_dim = (orig_h_or_w + (2 * padding) - kernel) // stride + 1
    return new_dim

### Global Variables

In [7]:
with open('../dataset/categories.json', 'r') as j:
    categories = json.load(j)    

cat_ids_we_want = [cat['id'] for cat in categories]
cat_ids_we_want
# catid_to_catname_df = pd.DataFrame.from_records(pd.Series([c.values() for c in categories])).drop(0, axis=1)
# catid_to_catname_df.to_csv('../dataset/coco_labels.txt', header=None, sep=',', mode='a')

[16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61]

In [8]:
train_np_data_dir = '../data/numpy_imgs/train_subset/'
train_jpg_data_dir = '../data/raw/train/train2014/'
train_annot_filepath = '../data/raw/train/annotations/instances_train2014.json'

with open('../dataset/imgs_by_supercategory.json', 'r') as f:
    desired_categories = json.load(f)

In [9]:
val_np_data_dir = '../data/numpy_imgs/val_subset/'
val_jpg_data_dir = '../data/raw/val/val2014/'
val_annot_filepath = '../data/raw/train/annotations/instances_val2014.json'

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')

### Load Data : Train & Validation into DataLoader

In [11]:
# torch.cuda.empty_cache()

In [12]:
%load_ext autoreload

In [13]:
%autoreload

In [14]:
train_ds = coco_data_prep.COCODataset('train',
                                      train_np_data_dir, 
                                      train_annot_filepath,
                                      coco_data_prep.COCOAnnotationTransform(),
                                      sample_ratio=0.05,
                                      device='cpu')

loading annotations into memory...
Done (t=11.69s)
creating index...
index created!


100%|███████████████████████████████████████████| 82783/82783 [00:47<00:00, 1731.58it/s]


In [15]:
train_dl = coco_data_prep.get_dataloader(train_ds, 
                                         batch_size=512, 
                                         device='cpu')

In [16]:
for i, (batch) in tqdm(enumerate(train_dl)):
    image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]

    image_batch = torch.stack(image_batch).to('cpu')
    label_batch = torch.FloatTensor([l[0] for l in label_batch])
    
    print(image_batch.shape)
    print(label_batch.shape)
    

1it [00:04,  4.54s/it]

torch.Size([512, 3, 224, 224])
torch.Size([512])


2it [00:04,  2.06s/it]

torch.Size([512, 3, 224, 224])
torch.Size([512])


3it [00:05,  1.22s/it]

...

7it [00:05,  2.49it/s]

torch.Size([512, 3, 224, 224])
torch.Size([512])
torch.Size([512, 3, 224, 224])
torch.Size([512])


9it [00:05,  3.45it/s]

torch.Size([43, 3, 224, 224])
torch.Size([43])


9it [00:06,  1.44it/s]


In [17]:
label_batch

tensor([16., 34., 17., 41., 99., 18., 39., 36., 99., 55., 99., 34., 34., 37.,
        99., 36., 24., 99., 20., 60., 99., 99., 36., 56., 56., 20., 99., 36.,
        17., 19., 20., 19., 38., 37., 39., 41., 39., 17., 20., 21., 37., 36.,
        38.])

In [ ]:
len(label_batch)

In [ ]:
len([[[60], [54], [99], [36], [99], [99], [99], [36], [38], [35], [99], [53], [61], [34], [53], [42], [53], [18], [99], [23], [37], [38], [99], [60], [16], [20], [19], [17], [35], [16], [99], [53], [16], [39], [99], [36], [38], [24], [21], [99], [23], [99], [59], [56], [58], [17], [18], [23], [38], [99], [36], [36], [36], [24], [99], [99], [57], [38], [38], [99], [37], [60], [34], [99], [42], [19], [35], [21], [56], [99], [99], [41], [36], [20], [99], [99], [56], [60], [16], [42], [38], [99], [16], [99], [20], [24], [42], [99], [17], [17], [99], [99], [38], [16], [99], [99], [99], [99], [24], [25], [99], [99], [99], [36], [16], [53], [60], [60], [99], [56], [61], [37], [41], [37], [34], [17], [55], [18], [55], [99], [99], [20], [20], [23], [40], [99], [21], [99], [25], [99], [40], [99], [16], [34], [36], [16], [20], [34], [35], [19], [99], [55], [41], [56], [52], [21], [34], [20], [16], [99], [16], [60], [17], [99], [16], [99], [41], [24], [53], [20], [42], [61], [56], [19], [21], [16], [23], [99], [18], [56], [37], [99], [24], [38], [99], [99], [36], [60], [99], [23], [99], [60], [37], [59], [21], [59], [39], [37], [99], [56], [99], [22], [23], [56], [16], [37], [34], [17], [23], [35], [99], [99], [57], [56], [99], [34], [24], [22], [60], [16], [99], [19], [99], [99], [56], [41], [39], [36], [36], [99], [22], [60], [37], [23], [36], [60], [99], [99], [17], [19], [16], [16], [20], [36], [55], [38], [16], [60], [38], [23], [99], [99], [56], [38], [99], [99], [24], [99], [17], [58], [17], [16], [42], [22], [99], [99], [55], [99], [37], [18], [38], [41], [34], [35], [36], [25], [22], [99], [99], [23], [16], [38], [55], [99], [16], [99], [16], [23], [60], [99], [41], [99], [24], [17], [20], [54], [99], [36], [54], [21], [22], [37], [99], [99], [35], [35], [20], [16], [36], [17], [19], [17], [23], [21], [37], [23], [99], [99], [99], [38], [54], [17], [37], [18], [38], [20], [38], [41], [23], [34], [99], [18], [99], [17], [99], [19], [38], [56], [37], [16], [41], [99], [60], [20], [19], [99], [37], [17], [24], [34], [16], [99], [99], [35], [38], [37], [17], [39], [57], [99], [20], [16], [16], [59], [54], [19], [21], [36], [35], [16], [52], [19], [99], [36], [22], [99], [16], [38], [23], [99], [23], [20], [99], [99], [99], [36], [16], [53], [42], [37], [19], [99], [22], [42], [36], [34], [20], [38], [42], [56], [34], [55], [99], [99], [58], [42], [24], [17], [55], [20], [56], [99], [55], [39], [36], [59], [38], [37], [53], [34], [34], [39], [36], [99], [34], [17], [16], [60], [99], [23], [54], [99], [34], [99], [18], [16], [41], [99], [38], [38], [23], [42], [35], [99], [21], [52], [56], [36], [38], [17], [99], [37], [18], [16], [99], [99], [34], [22], [19], [18], [99], [41], [37], [99], [99], [38], [57], [99], [20], [17], [99], [36], [40], [38], [17], [99], [99], [17], [23], [16], [99], [23], [36], [99], [37], [22], [18], [39], [38], [99], [39], [99], [36], [20], [22], [23], [55], [17], [99], [99], [43], [34], [99], [22], [56], [99], [19], [43], [99], [99], [22], [16], [17], [36], [99], [21], [99], [17], [52], [24], [43], [43], [60], [54], [99], [36], [99], [99], [99], [36], [38], [35], [99], [53], [61], [34], [53], [42], [53], [18], [99], [23], [37], [38], [99], [60], [16], [20], [19], [17], [35], [16], [99], [53], [16], [39], [99], [36], [38], [24], [21], [99], [23], [99], [59], [56], [58], [17], [18], [23], [38], [99], [36], [36], [36], [24], [99], [99], [57], [38], [38], [99], [37], [60], [34], [99], [42], [19], [35], [21], [56], [99], [99], [41], [36], [20], [99], [99], [56], [60], [16], [42], [38], [99], [16], [99], [20], [24], [42], [99], [17], [17], [99], [99], [38], [16], [99], [99], [99], [99], [24], [25], [99], [99], [99], [36], [16], [53], [60], [60], [99], [56], [61], [37], [41], [37], [34], [17], [55], [18], [55], [99], [99], [20], [20], [23], [40], [99], [21], [99], [25], [99], [40], [99], [16], [34], [36], [16], [20], [34], [35], [19], [99], [55], [41], [56], [52], [21], [34], [20], [16], [99], [16], [60], [17], [99], [16], [99], [41], [24], [53], [20], [42], [61], [56], [19], [21], [16], [23], [99], [18], [56], [37], [99], [24], [38], [99], [99], [36], [60], [99], [23], [99], [60], [37], [59], [21], [59], [39], [37], [99], [56], [99], [22], [23], [56], [16], [37], [34], [17], [23], [35], [99], [99], [57], [56], [99], [34], [24], [22], [60], [16], [99], [19], [99], [99], [56], [41], [39], [36], [36], [99], [22], [60], [37], [23], [36], [60], [99], [99], [17], [19], [16], [16], [20], [36], [55], [38], [16], [60], [38], [23], [99], [99], [56], [38], [99], [99], [24], [99], [17], [58], [17], [16], [42], [22], [99], [99], [55], [99], [37], [18], [38], [41], [34], [35], [36], [25], [22], [99], [99], [23], [16], [38], [55], [99], [16], [99], [16], [23], [60], [99], [41], [99], [24], [17], [20], [54], [99], [36], [54], [21], [22], [37], [99], [99], [35], [35], [20], [16], [36], [17], [19], [17], [23], [21], [37], [23], [99], [99], [99], [38], [54], [17], [37], [18], [38], [20], [38], [41], [23], [34], [99], [18], [99], [17], [99], [19], [38], [56], [37], [16], [41], [99], [60], [20], [19], [99], [37], [17], [24], [34], [16], [99], [99], [35], [38], [37], [17], [39], [57], [99], [20], [16], [16], [59], [54], [19], [21], [36], [35], [16], [52], [19], [99], [36], [22], [99], [16], [38], [23], [99], [23], [20], [99], [99], [99], [36], [16], [53], [42], [37], [19], [99], [22], [42], [36], [34], [20], [38], [42], [56], [34], [55], [99], [99], [58], [42], [24], [17], [55], [20], [56], [99], [55], [39], [36], [59], [38], [37], [53], [34], [34], [39], [36], [99], [34], [17], [16], [60], [99], [23], [54], [99], [34], [99], [18], [16], [41], [99], [38], [38], [23], [42], [35], [99], [21], [52], [56], [36], [38], [17], [99], [37], [18], [16], [99], [99], [34], [22], [19], [18], [99], [41], [37], [99], [99], [38], [57], [99], [20], [17], [99], [36], [40], [38], [17], [99], [99], [17], [23], [16], [99], [23], [36], [99], [37], [22], [18], [39], [38], [99], [39], [99], [36], [20], [22], [23], [55], [17], [99], [99], [43], [34], [99], [22], [56], [99], [19], [43], [99], [99], [22], [16], [17], [36], [99], [21], [99], [17], [52], [24], [43], [43]]][0])

In [ ]:
# val_ds = coco_data_prep.COCODataset('val',
#                                     val_np_data_dir, 
#                                     val_annot_filepath,
#                                     coco_data_prep.COCOAnnotationTransform(),
#                                     sample_ratio = 0.05,
#                                     device='cpu')

In [ ]:
# val_dl = coco_data_prep.get_dataloader(val_ds, 
#                                      batch_size=512, 
#                                      device='cpu')

### Load Model

#### Densenet121

In [ ]:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)

In [ ]:
# model

### Build From Scratch - Densenet Inspired

In [ ]:
class DensenetInspired(nn.Module):
    def __init__(self):
        super(DensenetInspired, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu1 = nn.ReLU()
        self.mp1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        
        # This part is Densenet inspired...recycle use of BN, RELU, CONV repeatedly
        self.bn2 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        
        self.bn3 = nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu3= nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        
        self.flat = nn.Flatten()
        self.lin1 = nn.LazyLinear(out_features=1024)
        self.fc1 = nn.Linear(1024, 30)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.mp1(out)
        
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.conv2(out)
        
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.conv3(out)
        
        out = self.flat(out)
        out = self.lin1(out)
        out = self.fc1(out)

        return out

#### Modeling Variables

In [ ]:
model = DensenetInspired()

In [ ]:
optim_params= dict(
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-05,
    weight_decay=0.01,
    amsgrad=False,
)

optimizer = torch.optim.Adam(model.parameters(), **optim_params)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
criterion = nn.MSELoss()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%autoreload

In [ ]:
import statistics

In [ ]:
[statistics.mode([20, 21, 21, 45, 45, 21, 23, 23, 23, 23])]

In [ ]:
for epoch in range(50):
    running_loss = 0.0
    count = 0
    
    for i, (batch) in tqdm(enumerate(train_dl)):
        image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]
        image_batch = torch.stack(image_batch).to('cpu')
        label_batch = torch.stack(label_batch).to('cpu')
        
#         try:
#             optimizer.zero_grad()
#             outputs = model(image_batch)
            
#             loss = criterion(outputs, label_batch.double())
#             print(loss)
            
#             loss.backward()
#             optimizer.step()
#             count += 1
#             running_loss += loss.item()
#             scheduler.step()
            
#             if count % 100 == 99:    # print every 10 mini-batches
#                 print('[%d, %5d] loss: %.3f' %
#                         (epoch + 1, count + 1, running_loss / 100))
#                 running_loss = 0.0
#         except:
#             continue
# #             print('not enough in batch_size')
#     print(f'Epoch-{epoch} lr: {optimizer.param_groups[0]["lr"]} running_loss: {running_loss}')

In [ ]:
running_loss

In [ ]:
features_list = []
labels_list = []

# Use GPUs to speed up the inference, this should take around 10 minutes

model.to('cpu')
for batch in tqdm(train_dl):
    image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]
    image_batch = torch.stack(image_batch).to('cpu') 

    with torch.no_grad():
        features_batch = model(image_batch) # .flatten(start_dim=1)
    features_list.append(features_batch)
    labels_list.extend(label_batch)

In [ ]:
type(train_ds[0][1])

In [ ]:
criterion(features_list[0], [44, 44, 59, 1, 44, 44, 79, 79])

In [ ]:
[[l['category_id'] for l in label] for label in labels_list]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
features_list[0].shape

In [ ]:
for i, vecs in enumerate(features_list):
    torch.save(vecs, f'../data/torch_embeddings/densenet_pretrained_embs_len_50176-{i}.pt')

In [ ]:
(0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
(1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
(2): ReLU(inplace=True)
(3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)

In [ ]:
class DensenetInspired(nn.Module):
    def __init__(self):
        super(DensenetInspired).__init__()
        
        self.conv0 = model.features.conv0
        self.bn0 = model.features.norm0
        self.relu0 = model.features.relu0
        self.pool0 = model.features.pool0
        self.db0 = nn.BatchNorm2d(64, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(64, 32, kernel_size(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        self.linear = nn.Linear(in_features=1024, out_features=1000, bias=True)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x